In [1]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
import joblib
from scipy.special import softmax
import sasc.config
import numpy as np
from collections import defaultdict
from copy import deepcopy
import pandas as pd
import sasc.viz
from sasc import analyze_helper
from sasc.modules.fmri_module import convert_module_num_to_voxel_num
from sasc.config import FMRI_DIR, RESULTS_DIR
import dvu
dvu.set_style()

[2023-11-21 15:15:00,370] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/thinc/compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/thinc/compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]


### Load pilot pickle

In [3]:
stories_data_dict = joblib.load(
    join(sasc.config.RESULTS_DIR, 'pilot_story_data.pkl'))
pilot_data_dir = '/home/chansingh/mntv1/deep-fMRI/story_data/20230504'

# stories_data_dict = joblib.load(
#     join(sasc.config.RESULTS_DIR, 'pilot3_story_data.pkl'))
# pilot_data_dir = '/home/chansingh/mntv1/deep-fMRI/story_data/20231106'

### Read all the info from stories into a single pickle file

In [4]:
# load responses
default_story_idxs = np.where(
    np.array(stories_data_dict['story_setting']) == 'default')[0]
resp_np_files = [stories_data_dict['story_name_new'][i].replace('_resps', '')
                 for i in default_story_idxs]
resps_dict = {
    k: np.load(join(pilot_data_dir, k))
    for k in tqdm(resp_np_files)
}

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:29<00:00,  5.00s/it]


### Let's check the alignment

In [9]:
timings_list = stories_data_dict['timing']
story_names_list = list(resps_dict.keys())
resps = list(resps_dict.values())

In [20]:
TRIM = 5
for i in range(len(resps)):
    t = timings_list[i]
    duration_secs = t['time_running'].max()
    print(story_names_list[i], 'resp_length',
          resps[i].shape[0], 'story_trs',
          int(duration_secs // 2), 'story_length', duration_secs)  # , 'timings',
    diff = int(duration_secs // 2) - resps[i].shape[0]
    assert abs(diff - TRIM * 2) <= 1

GenStory1.npy resp_length 435 story_trs 445 story_length 890.2769680636738
GenStory2.npy resp_length 382 story_trs 392 story_length 784.0274450678589
GenStory3.npy resp_length 322 story_trs 331 story_length 663.8185519243381
GenStory4.npy resp_length 405 story_trs 415 story_length 830.2452811179473
GenStory5.npy resp_length 407 story_trs 417 story_length 834.1917651885974
GenStory6.npy resp_length 470 story_trs 480 story_length 960.9583298519948


### Let's check the paragraph<>timing match